# 서울 구별 주택공급 현황 분석

이 노트북은 주택공급 데이터 파일에서 서울의 25개 구 공급현황을 추출하고 분석합니다.

## 1. 환경 설정

In [1]:
import sys
sys.path.append("..")

import pandas as pd
from pathlib import Path

print("✅ 환경 설정 완료")

✅ 환경 설정 완료


## 2. 샘플 데이터 생성

In [2]:
# 샘플 데이터 생성
sample_data = pd.DataFrame({
    "시군구": [
        "서울특별시 강남구", "서울특별시 서초구", "서울특별시 송파구",
        "서울특별시 강동구", "서울특별시 마포구", "서울특별시 용산구",
        "경기도 성남시", "경기도 수원시",
        "서울특별시 강서구", "서울특별시 양천구"
    ],
    "연도": [2024] * 10,
    "공급량": [1000, 800, 1200, 600, 900, 700, 1500, 1100, 850, 750]
})

SAMPLE_FILE = "sample_supply_data.xlsx"
sample_data.to_excel(SAMPLE_FILE, index=False)

print("📝 샘플 데이터 생성 완료")
sample_data

📝 샘플 데이터 생성 완료


,시군구,연도,공급량
0,서울특별시 강남구,2024,1000
1,서울특별시 서초구,2024,800
2,서울특별시 송파구,2024,1200
3,서울특별시 강동구,2024,600
4,서울특별시 마포구,2024,900
5,서울특별시 용산구,2024,700
6,경기도 성남시,2024,1500
7,경기도 수원시,2024,1100
8,서울특별시 강서구,2024,850
9,서울특별시 양천구,2024,750


## 3. HousingSupplyTool 사용

In [3]:
from tools.housing_supply_tool import HousingSupplyTool

supply_tool = HousingSupplyTool(SAMPLE_FILE)
print(f"데이터 크기: {supply_tool.df.shape}")

📁 데이터 로드 중: sample_supply_data.xlsx
✅ 데이터 로드 완료: 10행 × 3열
컬럼: ['시군구', '연도', '공급량']
데이터 크기: (10, 3)


## 4. 서울 25개 구만 필터링

In [4]:
seoul_df = supply_tool.filter_seoul_districts(district_column="시군구")
print(f"서울 구 데이터: {len(seoul_df)}건")
seoul_df

✅ 서울 구 필터링 완료: 8개 구
서울 구 데이터: 8건


,시군구,연도,공급량
0,서울특별시 강남구,2024,1000
1,서울특별시 서초구,2024,800
2,서울특별시 송파구,2024,1200
3,서울특별시 강동구,2024,600
4,서울특별시 마포구,2024,900
5,서울특별시 용산구,2024,700
8,서울특별시 강서구,2024,850
9,서울특별시 양천구,2024,750


## 5. 실제 데이터 파일 분석 (연도별/월별)

In [18]:
from bs4 import BeautifulSoup
# 실제 데이터 파일 경로
REAL_DATA_FILE = r"C:\RAG_COMMANDER\src\lab\주택 10년 공급현황.xls"

# XML 파일 열기 (EUC-KR 인코딩)
with open(REAL_DATA_FILE, "r", encoding="EUC-KR", errors="ignore") as f:
    soup = BeautifulSoup(f, "xml")

# 모든 Row 추출
rows = []
for row in soup.find_all("Row"):
    cells = [cell.get_text(strip=True) for cell in row.find_all("Data")]
    if any(cells):  # 빈 행 제외
        rows.append(cells)

# DataFrame으로 변환
real_df = pd.DataFrame(rows)
real_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,"○ 주택종류별 주택- 읍면동(연도 끝자리 0,5), 시군구(그 외 연도) [호]",None,None,None,None,None,None,None,None,None,None,None,None,None
1,동별,주택종류별,항목,단위,2015 년,2016 년,2017 년,2018 년,2019 년,2020 년,2021 년,2022 년,2023 년,2024 년
2,서울시,종류별 주택수,주택종류별 주택 (동별),,2793244,2830857,2866845,2894078,2953964,3015371,3068494,3111323,3155331,3170332
3,서울시,단독주택,주택종류별 주택 (동별),,355039,345269,331863,322453,315566,307075,299473,293489,291582,286542
4,서울시,일반단독주택,주택종류별 주택 (동별),,88163,84680,78193,73510,68114,64911,62648,60861,59966,58667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,강동구,영업겸용주택,주택종류별 주택 (동별),,2077,2116,2140,2163,2189,2178,2177,2192,2219,2202
232,강동구,아파트,주택종류별 주택 (동별),,78697,71613,75263,75473,73505,87159,94531,95721,97890,101647
233,강동구,연립주택,주택종류별 주택 (동별),,3704,3640,3551,3512,3165,3114,3199,3216,3252,3377
234,강동구,다세대주택,주택종류별 주택 (동별),,24886,28645,30251,32149,35001,36562,37790,38934,39584,39881


### 5.1 데이터 전처리 및 날짜 변환